In [1]:
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
from scipy.stats import pearsonr
from statsmodels.tsa.arima_model import ARIMA
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from datetime import timedelta
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten, Dropout, Activation

from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

handler = logging.FileHandler('nn.log')
handler.setLevel(logging.INFO)
logger.addHandler(handler)

%matplotlib inline
init_notebook_mode(connected = True)

Using TensorFlow backend.
/home/rbkn99/.local/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning:

inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()



In [4]:
class PPPredictor:
    
    def __init__(self):
        self.model = None
    
    def prepare_data(self, data, actual_date):
        df = data.copy()
        df.index = pd.to_datetime(df['Date'])
        df.drop(['Date'], axis=1, inplace=True)
        for col in df.columns:
            df[col].interpolate(method='time', inplace=True)
        df.index = pd.to_datetime(df.index)
        return df[df.index <= pd.to_datetime(actual_date)]

    def fit(self, prepared_data, use_text_model=False):
        model = Sequential()

        model.add(LSTM(
            input_dim=1,
            output_dim=50,
            return_sequences=True))
        model.add(Dropout(0.2))

        model.add(LSTM(
            100,
            return_sequences=False))
        model.add(Dropout(0.2))

        model.add(Dense(
            output_dim=1))
        model.add(Activation('linear'))

        model.compile(loss='mse', optimizer='rmsprop')
        
        df = pd.DataFrame({'ds': prepared_data.index, 'y': prepared_data['PPSpotAvgPrice']}).reset_index().drop(['Date'], axis=1)
        df['ds'] = pd.to_numeric(df['ds'])
        
        self.scaler_X = MinMaxScaler()
        X_train = np.reshape(np.array(df['ds']), (-1, 1))
        X_train = self.scaler_X.fit_transform(X_train)
        
        self.scaler_y = MinMaxScaler()
        y_train = np.reshape(np.array(df['y']), (-1, 1))
        y_train = self.scaler_y.fit_transform(y_train)
        
        X_train = np.reshape(X_train, (X_train.shape[0], 1, 1))
        #y_train = np.reshape(y_train, (y_train.shape[0], 1, 1))
        model.fit(X_train, y_train, batch_size=128, nb_epoch=10, validation_split=0.05)
        
        self.model = model
        return model

    
    def predict(self, date):
        X_test = np.array(pd.to_numeric(pd.Series(pd.to_datetime(date))))
        X_test = np.reshape(X_test, (-1, 1))
        X_test = self.scaler_X.transform(X_test)
        X_test = np.reshape(X_test, (1, 1, 1))
        forecast = self.model.predict(X_test)
        return self.scaler_y.inverse_transform(forecast)

In [5]:
 def get_next_monday(df, date):
    first = True
    while len(df[df.index == str(date).split()[0]]) == 0:
        if first:
            date += timedelta(days=((7 - date.weekday()) % 7))
            first = False
        else:
            date += timedelta(7)
    return date

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def test_sol(date):
    df = pd.read_csv('./data/retrieved_data.csv')
    ppp = PPPredictor()
    prepared = ppp.prepare_data(df, date)
    ppp.fit(prepared)
    
    df.index = pd.to_datetime(df['Date'])
    df.drop(['Date'], axis=1, inplace=True)
    for col in df.columns:
        df[col].interpolate(method='time', inplace=True)
    
    start = pd.to_datetime(date) + timedelta(3 * 30)
    end = pd.to_datetime(date) + timedelta(4 * 30)
    pred = []
    actual = []
    for dt in pd.date_range(start, end, freq='W'):
        pred.append(ppp.predict(str(dt).split()[0]))
        actual_dt = get_next_monday(df, dt)
        right_val = df[df.index == str(actual_dt).split()[0]]['PPSpotAvgPrice'].iloc[0]
        actual.append(right_val)
    logger.info('MAPE for {}: {:.2f}%'.format(date, mean_absolute_percentage_error(pred, actual)))

for dt in pd.date_range(start=pd.to_datetime('2017-12-01'), 
                        end=pd.to_datetime('2018-07-01'), freq='M'):
    test_sol(str(dt).split()[0])

Train on 201 samples, validate on 11 samples
Epoch 1/10
201/201 [==============================] - 4s 19ms/step - loss: 0.2669 - val_loss: 0.1867
Epoch 2/10
201/201 [==============================] - 0s 81us/step - loss: 0.2442 - val_loss: 0.1616
Epoch 3/10
201/201 [==============================] - 0s 87us/step - loss: 0.2272 - val_loss: 0.1396
Epoch 4/10
201/201 [==============================] - 0s 77us/step - loss: 0.2120 - val_loss: 0.1188
Epoch 5/10
201/201 [==============================] - 0s 99us/step - loss: 0.1979 - val_loss: 0.0992
Epoch 6/10
201/201 [==============================] - 0s 57us/step - loss: 0.1843 - val_loss: 0.0802
Epoch 7/10
201/201 [==============================] - 0s 92us/step - loss: 0.1717 - val_loss: 0.0635
Epoch 8/10
201/201 [==============================] - 0s 59us/step - loss: 0.1608 - val_loss: 0.0487
Epoch 9/10
201/201 [==============================] - 0s 89us/step - loss: 0.1494 - val_loss: 0.0353
Epoch 10/10
201/201 [=========================

INFO:__main__:MAPE for 2017-12-31: 8.12%


Train on 206 samples, validate on 11 samples
Epoch 1/10
206/206 [==============================] - 4s 18ms/step - loss: 0.2637 - val_loss: 0.2089
Epoch 2/10
206/206 [==============================] - 0s 66us/step - loss: 0.2385 - val_loss: 0.1795
Epoch 3/10
206/206 [==============================] - 0s 47us/step - loss: 0.2198 - val_loss: 0.1532
Epoch 4/10
206/206 [==============================] - 0s 83us/step - loss: 0.2035 - val_loss: 0.1288
Epoch 5/10
206/206 [==============================] - 0s 81us/step - loss: 0.1882 - val_loss: 0.1060
Epoch 6/10
206/206 [==============================] - 0s 96us/step - loss: 0.1738 - val_loss: 0.0848
Epoch 7/10
206/206 [==============================] - 0s 89us/step - loss: 0.1600 - val_loss: 0.0656
Epoch 8/10
206/206 [==============================] - 0s 176us/step - loss: 0.1488 - val_loss: 0.0499
Epoch 9/10
206/206 [==============================] - 0s 117us/step - loss: 0.1380 - val_loss: 0.0356
Epoch 10/10
206/206 [=======================

INFO:__main__:MAPE for 2018-01-31: 9.34%


Train on 209 samples, validate on 12 samples
Epoch 1/10
209/209 [==============================] - 4s 20ms/step - loss: 0.2636 - val_loss: 0.2204
Epoch 2/10
209/209 [==============================] - 0s 49us/step - loss: 0.2393 - val_loss: 0.1907
Epoch 3/10
209/209 [==============================] - 0s 56us/step - loss: 0.2212 - val_loss: 0.1639
Epoch 4/10
209/209 [==============================] - 0s 61us/step - loss: 0.2048 - val_loss: 0.1389
Epoch 5/10
209/209 [==============================] - 0s 77us/step - loss: 0.1893 - val_loss: 0.1155
Epoch 6/10
209/209 [==============================] - 0s 76us/step - loss: 0.1746 - val_loss: 0.0937
Epoch 7/10
209/209 [==============================] - 0s 90us/step - loss: 0.1620 - val_loss: 0.0740
Epoch 8/10
209/209 [==============================] - 0s 142us/step - loss: 0.1496 - val_loss: 0.0564
Epoch 9/10
209/209 [==============================] - 0s 103us/step - loss: 0.1377 - val_loss: 0.0413
Epoch 10/10
209/209 [=======================

...

INFO:__main__:MAPE for 2018-04-30: 16.75%


Train on 222 samples, validate on 12 samples
Epoch 1/10
222/222 [==============================] - 5s 24ms/step - loss: 0.2591 - val_loss: 0.1636
Epoch 2/10
222/222 [==============================] - 0s 69us/step - loss: 0.2346 - val_loss: 0.1380
Epoch 3/10
222/222 [==============================] - 0s 64us/step - loss: 0.2170 - val_loss: 0.1158
Epoch 4/10
222/222 [==============================] - 0s 78us/step - loss: 0.1997 - val_loss: 0.0948
Epoch 5/10
222/222 [==============================] - 0s 88us/step - loss: 0.1848 - val_loss: 0.0755
Epoch 6/10
222/222 [==============================] - 0s 108us/step - loss: 0.1702 - val_loss: 0.0580
Epoch 7/10
222/222 [==============================] - 0s 71us/step - loss: 0.1561 - val_loss: 0.0424
Epoch 8/10
222/222 [==============================] - 0s 97us/step - loss: 0.1428 - val_loss: 0.0291
Epoch 9/10
222/222 [==============================] - 0s 82us/step - loss: 0.1321 - val_loss: 0.0183
Epoch 10/10
222/222 [========================

INFO:__main__:MAPE for 2018-05-31: 18.07%


Train on 226 samples, validate on 12 samples
Epoch 1/10
226/226 [==============================] - 6s 25ms/step - loss: 0.2627 - val_loss: 0.1970
Epoch 2/10
226/226 [==============================] - 0s 48us/step - loss: 0.2374 - val_loss: 0.1694
Epoch 3/10
226/226 [==============================] - 0s 56us/step - loss: 0.2184 - val_loss: 0.1445
Epoch 4/10
226/226 [==============================] - 0s 71us/step - loss: 0.2017 - val_loss: 0.1208
Epoch 5/10
226/226 [==============================] - 0s 75us/step - loss: 0.1845 - val_loss: 0.0987
Epoch 6/10
226/226 [==============================] - 0s 78us/step - loss: 0.1698 - val_loss: 0.0783
Epoch 7/10
226/226 [==============================] - 0s 79us/step - loss: 0.1558 - val_loss: 0.0598
Epoch 8/10
226/226 [==============================] - 0s 162us/step - loss: 0.1428 - val_loss: 0.0439
Epoch 9/10
226/226 [==============================] - 0s 124us/step - loss: 0.1322 - val_loss: 0.0305
Epoch 10/10
226/226 [=======================

INFO:__main__:MAPE for 2018-06-30: 29.28%
